In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql
from sqlalchemy import create_engine

In [2]:
df = pd.read_csv('Candidate Performance.csv')
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [4]:
df = df.rename(columns = {'race/ethnicity': 'race_ethnicity',
                          'parental level of education': 'parental_level_of_education',
                          'test preparation course': 'test_preparation_course',
                          'math score': 'math_score',
                          'reading score': 'reading_score',
                          'writing score': 'writing_score'})
df

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [5]:
engine = create_engine('sqlite://', echo = False)

# export the dataframe as a table to the sqlite engine
df.to_sql('candidates', con = engine)

1000

In [6]:
# select some entries from table to test the code

sql='''

select *
from candidates
limit 5

''';

    
df_sql = pd.read_sql_query(sql, con = engine)
df_sql

,index,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,0,female,group B,bachelor's degree,standard,none,72,72,74
1,1,female,group C,some college,standard,completed,69,90,88
2,2,female,group B,master's degree,standard,none,90,95,93
3,3,male,group A,associate's degree,free/reduced,none,47,57,44
4,4,male,group C,some college,standard,none,76,78,75


In [7]:
# top 10 performers

sql='''

with data as
(select 
ROW_NUMBER() OVER () as id,
math_score+reading_score+writing_score as total_score
from candidates)

select id, total_score
from data
order by total_score desc, id asc
limit 10

''';

    
df_sql = pd.read_sql_query(sql, con = engine)
df_sql

,id,total_score
0,459,300
1,917,300
2,963,300
3,115,299
4,180,297
5,713,297
6,166,296
7,626,296
8,150,293
9,686,293


In [8]:
# bottom 10 performers

sql='''

with data as
(select 
ROW_NUMBER() OVER () as id,
math_score + reading_score + writing_score as total_score
from candidates)

select id, total_score
from data
order by total_score asc, id asc
limit 10

''';

    
df_sql = pd.read_sql_query(sql, con = engine)
df_sql

,id,total_score
0,60,27
1,981,55
2,597,69
3,328,70
4,18,78
5,77,78
6,602,88
7,339,89
8,788,89
9,212,90


In [9]:
# count of above average candidates

sql='''

with data as
(select 
ROW_NUMBER() OVER () as id, gender, race_ethnicity,
math_score + reading_score + writing_score as total_score
from candidates)

select gender, count(1) as number_of_candidates, count(distinct race_ethnicity) as count_of_races
from data
where total_score > (select avg(total_score) from data)
group by gender

''';

    
df_sql = pd.read_sql_query(sql, con = engine)
df_sql

,gender,number_of_candidates,count_of_races
0,female,300,5
1,male,226,5


In [10]:
# count of below average candidates

sql='''

with data as
(select 
ROW_NUMBER() OVER () as id, gender, race_ethnicity,
math_score + reading_score + writing_score as total_score
from candidates)

select gender, count(1) as number_of_candidates, count(distinct race_ethnicity) as count_of_races
from data
where total_score <= (select avg(total_score) from data)
group by gender

''';

    
df_sql = pd.read_sql_query(sql, con = engine)
df_sql

,gender,number_of_candidates,count_of_races
0,female,218,5
1,male,256,5


In [11]:
# we can join the above two tables or use this alternate approach

sql='''

with data as
(select 
ROW_NUMBER() OVER () as id, gender, race_ethnicity,
math_score + reading_score + writing_score as total_score
from candidates)

select gender,
sum(case when total_score > (select avg(total_score) from data) then 1 else 0 end) as above_avg,
sum(case when total_score <= (select avg(total_score) from data) then 1 else 0 end) as below_avg
from data
group by gender

''';

    
df_sql = pd.read_sql_query(sql, con = engine)
df_sql

,gender,above_avg,below_avg
0,female,300,218
1,male,226,256


In [12]:
# count of distinct races

sql='''


select gender, count(distinct race_ethnicity) as count_of_races
from candidates
group by gender

''';

    
df_sql = pd.read_sql_query(sql, con = engine)
df_sql

,gender,count_of_races
0,female,5
1,male,5


In [13]:
# count of candidates whose parents doesnt have a college degree

sql='''

select gender, count(1) as count_of_candidates
from candidates
where parental_level_of_education in ('high school', 'some high school')
group by gender

''';

    
df_sql = pd.read_sql_query(sql, con = engine)
df_sql

,gender,count_of_candidates
0,female,185
1,male,190


In [14]:
# candidates and their test preparation

sql='''

select gender,
sum(case when test_preparation_course = 'completed' then 1 else 0 end) as test_prep_complete,
sum(case when test_preparation_course <> 'completed' then 1 else 0 end) as test_prep_incomplete
from candidates
group by gender

''';

    
df_sql = pd.read_sql_query(sql, con = engine)
df_sql

,gender,test_prep_complete,test_prep_incomplete
0,female,184,334
1,male,174,308
